In [2]:
from config import *

# Prompt Templates

### Prompt Template

In [3]:
from langchain import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a {adjective} joke about {content}."
)
prompt_template.format(adjective="funny", content="cats")

'Tell me a funny joke about cats.'

In [ ]:
# a template withou
from langchain import PromptTemplate

prompt_template = PromptTemplate.from_template(
    "Tell me a joke"
)
prompt_template.format()

In [6]:
# for additional validation, specify input_variables explicitly

from langchain import PromptTemplate

valid_prompt = PromptTemplate(
    template="Tell me a {adjective} joke about {content}.",
    input_variables=["adjective", "content"]
)

In [ ]:
# for additional validation, specify input_variables explicitly

from langchain import PromptTemplate

invalid_prompt = PromptTemplate(
    template="Tell me a {adjective} joke about {content}.",
    input_variables=["adjective"]
)

### Chat prompt template

In [9]:
from langchain.prompts import ChatPromptTemplate

template = ChatPromptTemplate.from_messages([
    ('system', "You are a helpful AI bot. Your name is {name}."),
    ('human','Hello, how are you doing?'),
    ('ai', 'I am doing well, how are you?'),
    ('human', '{user_input}'),
])

messages = template.format_messages(
    name='Bob',
    user_input='What is your name?'
)

In [10]:
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.chat import SystemMessage, HumanMessagePromptTemplate

template = ChatPromptTemplate.from_messages(
    [
        SystemMessage(
            content=("You are a helpful assistant that re-writes the user's text to "
            "sound more upbeat."
            )
        ),
        HumanMessagePromptTemplate.from_template("{text}"),
    ]
)

from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI()
llm(template.format_messages(text='i dont like eating tasty tings.'))

AIMessage(content='I absolutely adore indulging in delicious treats!', additional_kwargs={}, example=False)

## Connecting to a Feature Store

### Feast

# Custom prompt template

### Creating a custom prompt template

In [16]:
import inspect
def get_source_code(function_name):
    return inspect.getsource(function_name)

In [17]:
from langchain.prompts import StringPromptTemplate
from pydantic import BaseModel, validator

In [18]:
PROMPT = """\
    Given the function name and source code, generate an English language explanation of the function.
    Function Name: {function_name}
    Source Code: 
    {source_code}
    Expalanation:
    """

In [20]:
class FunctionExplainerPromptTemplate(StringPromptTemplate, BaseModel):
    """A custom prompt template that takes in the fucntion name as input, and formats the prompt template to provide the source code of the function."""
    
    @validator("input_variables")
    def validate_input_variables(cls, v):
        """Validate that the input variables are correct."""
        if len(v) != 1 or "function_name" not in v:
            raise ValueError("function_name must be the only input variable.")
        return v
    
    def format(self, **kwargs) -> str:
        # Get the source code of the function
        source_code  = get_source_code(kwargs["function_name"])
        
        # Generate the prompt to be sent to the language model
        prompt = PROMPT.format(
            function_name=kwargs["function_name"].__name__, source_code=source_code
        )
        return prompt
    
    def _prompt_type(self) -> str:
        return super()._prompt_type() + "function_explainer"

### Use the custom prompt template


In [22]:
fn_explainer = FunctionExplainerPromptTemplate(input_variables=["function_name"])

# Generate a prompt for the function "get_source_code"ArithmeticError
prompt = fn_explainer.format(function_name=get_source_code)

In [24]:
print(prompt)

    Given the function name and source code, generate an English language explanation of the function.
    Function Name: get_source_code
    Source Code: 
    def get_source_code(function_name):
    return inspect.getsource(function_name)

    Expalanation:
    


## Few-shot prompt templates

In [25]:
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.prompt import PromptTemplate

In [27]:
examples = [
    {
        "question": "who lived longer, Muhammad Ali or Alan Turing?",
        "answer": 
            """
            Are follow up questions needed here: Yes.
            Follow up: How old was Muhammad Ali when he died?
            Intermediate answer: Muhammad Ali was 74 years old when he died.
            Follow up: How old was Alan Turing when he died?
            Intermediate answer: Alan Turing was 41 years old when he died.
            So the final answer is: Muhammad Ali
            """
    },
    {
        "question": "When was the founder of craigslist born?",
        "answer":
            """
            Are the follow up questions needed here: Yes.
            Follow up: who was the founder of craigslist?
            Intermediate answer: Craig Newmark was the founder of craigslist.
            Follow up: When was Craig Newmark born?
            Intermediate answer: Craig Newmark was born on December 6, 1952.
            So the final answer is: December 6, 1952
            """
    },
    {
    "question": "Who was the maternal grandfather of George Washington?",
    "answer":
"""
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washington?
Intermediate answer: The father of Mary Ball Washington was Joseph Ball.
So the final answer is: Joseph Ball
"""
  },
  {
    "question": "Are both the directors of Jaws and Casino Royale from the same country?",
    "answer":
"""
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No
"""
  }
]

### Create a formatter for the few shot examples

In [28]:
example_prompt = PromptTemplate(input_variables=["question", "answer"], template="Question: {question}\nAnswer: {answer}\n\n")
print(example_prompt.format(**examples[0]))

Question: who lived longer, Muhammad Ali or Alan Turing?
Answer: 
            Are follow up questions needed here: Yes.
            Follow up: How old was Muhammad Ali when he died?
            Intermediate answer: Muhammad Ali was 74 years old when he died.
            Follow up: How old was Alan Turing when he died?
            Intermediate answer: Alan Turing was 41 years old when he died.
            So the final answer is: Muhammad Ali
            




### Feed examples and formatter to FewShotPromptTemplate 

In [29]:
prompt = FewShotPromptTemplate(
    examples = examples,
    example_prompt=example_prompt,
    suffix="Question:{input}\nAnswer:",
    input_variables=["input"],
)

In [31]:
print(prompt.format(input="Who is the President of Pakistan?"))

Question: who lived longer, Muhammad Ali or Alan Turing?
Answer: 
            Are follow up questions needed here: Yes.
            Follow up: How old was Muhammad Ali when he died?
            Intermediate answer: Muhammad Ali was 74 years old when he died.
            Follow up: How old was Alan Turing when he died?
            Intermediate answer: Alan Turing was 41 years old when he died.
            So the final answer is: Muhammad Ali
            



Question: When was the founder of craigslist born?
Answer: 
            Are the follow up questions needed here: Yes.
            Follow up: who was the founder of craigslist?
            Intermediate answer: Craig Newmark was the founder of craigslist.
            Follow up: When was Craig Newmark born?
            Intermediate answer: Craig Newmark was born on December 6, 1952.
            So the final answer is: December 6, 1952
            



Question: Who was the maternal grandfather of George Washington?
Answer: 
Are follow up

### Using an example selector

In [32]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

In [35]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    Chroma,
    k=2)

In [39]:
# Select the most similar example to the input.

question = "Who was the father of craigslist founder?"
selected_examples = example_selector.select_examples({'question': question})
print(selected_examples)
for example in selected_examples:
    print(example['question'])
    print(example['answer'])
    print()

[{'question': 'When was the founder of craigslist born?', 'answer': '\n            Are the follow up questions needed here: Yes.\n            Follow up: who was the founder of craigslist?\n            Intermediate answer: Craig Newmark was the founder of craigslist.\n            Follow up: When was Craig Newmark born?\n            Intermediate answer: Craig Newmark was born on December 6, 1952.\n            So the final answer is: December 6, 1952\n            '}, {'question': 'Who was the maternal grandfather of George Washington?', 'answer': '\nAre follow up questions needed here: Yes.\nFollow up: Who was the mother of George Washington?\nIntermediate answer: The mother of George Washington was Mary Ball Washington.\nFollow up: Who was the father of Mary Ball Washington?\nIntermediate answer: The father of Mary Ball Washington was Joseph Ball.\nSo the final answer is: Joseph Ball\n'}]
When was the founder of craigslist born?

            Are the follow up questions needed here: Yes.


### Feed example selector into FewShotPromptTemplate

In [42]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question:{question}\nAnswer:",
    input_variables=["question"],
)

print(prompt.format(question="Who is a fighter?"))

Question: Are both the directors of Jaws and Casino Royale from the same country?
Answer: 
Are follow up questions needed here: Yes.
Follow up: Who is the director of Jaws?
Intermediate Answer: The director of Jaws is Steven Spielberg.
Follow up: Where is Steven Spielberg from?
Intermediate Answer: The United States.
Follow up: Who is the director of Casino Royale?
Intermediate Answer: The director of Casino Royale is Martin Campbell.
Follow up: Where is Martin Campbell from?
Intermediate Answer: New Zealand.
So the final answer is: No




Question: who lived longer, Muhammad Ali or Alan Turing?
Answer: 
            Are follow up questions needed here: Yes.
            Follow up: How old was Muhammad Ali when he died?
            Intermediate answer: Muhammad Ali was 74 years old when he died.
            Follow up: How old was Alan Turing when he died?
            Intermediate answer: Alan Turing was 41 years old when he died.
            So the final answer is: Muhammad Ali
         

## Few shot examples for chat models

In [43]:
from langchain.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate,
)

In [61]:
examples = [
    {"input": "2+2", "output": "1"},
    {"input": "2+3", "output": "2"},
]

In [62]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human","{input}"),
        ("ai","{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

In [70]:
print(few_shot_prompt.format())

Human: 2+2
AI: 1
Human: 2+3
AI: 2


In [75]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are wonderous wizard of math. You have to give answer like you are giving in examples"),
        few_shot_prompt,
        ("human", "{input}")
    ]
)

In [76]:
from langchain.chat_models import ChatOpenAI

chain = final_prompt | ChatOpenAI(temperature=0.0)

In [77]:
print(final_prompt.format(input=""))

System: You are wonderous wizard of math. You have to give answer like you are giving in examples
Human: 2+2
AI: 1
Human: 2+3
AI: 2
Human: 


In [78]:
chain.invoke({"input": "2+4"})

AIMessage(content='3', additional_kwargs={}, example=False)

### Dynamic Few-shot Prompting

In [79]:
from langchain.prompts import SemanticSimilarityExampleSelector
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma

In [80]:
examples = [
    {"input": "2+2", "output": "4"},
    {"input": "2+3", "output": "5"},
    {"input": "2+4", "output": "6"},
    {"input": "What did the cow say to the moon?", "output": "nothing at all"},
    {
        "input": "Write me a poem about the moon",
        "output": "One for the moon, and one for me, who are we to talk about the moon?",
    },
]

In [81]:
to_vectorize = [" ".join(example.values()) for example in examples]
to_vectorize

['2+2 4',
 '2+3 5',
 '2+4 6',
 'What did the cow say to the moon? nothing at all',
 'Write me a poem about the moon One for the moon, and one for me, who are we to talk about the moon?']

In [84]:
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(to_vectorize, embeddings, metadatas=examples)

In [101]:
example_selector = SemanticSimilarityExampleSelector(
    vectorstore=vectorstore,
    k=2,
)

In [102]:
example_selector.select_examples({"input": "horse"})

[{'input': 'What did the cow say to the moon?', 'output': 'nothing at all'},
 {'input': '2+4', 'output': '6'}]

In [103]:
from langchain.prompts import (
    FewShotChatMessagePromptTemplate,
    ChatPromptTemplate,
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    input_variables=["input"],
    example_selector=example_selector,
    example_prompt=ChatPromptTemplate.from_messages(
        [(
            "human","{input}"
        ),
         (
             "ai","{output}"
         )]
    )
)

In [104]:
print(few_shot_prompt.format(input="What's 3+3?"))

Human: 2+3
AI: 5
Human: 2+2
AI: 4


In [105]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system","You are wonderous wizard of math"),
        few_shot_prompt,
        ("human","{input}"),
    ]
)

In [108]:
print(final_prompt.format(input="What is 2+5? "))

System: You are wonderous wizard of math
Human: 2+3
AI: 5
Human: 2+2
AI: 4
Human: What is 2+5? 


In [119]:
from langchain.chat_models import ChatOpenAI

chain  = final_prompt | ChatOpenAI(temperature=0.0)

chain.invoke({"input":"What's 3+3?"})

AIMessage(content='3 + 3 equals 6.', additional_kwargs={}, example=False)

# Format template output

# Template Formats

In [121]:
# to use jinja2 as template engine

from langchain.prompts import PromptTemplate

In [122]:
jinja2_template = "Tell me a {{ adjective }} joke about {{ content }}"

prompt =PromptTemplate.from_template(jinja2_template, template_format="jinja2")

prompt.format(adjective="funny", content="a dog")

'Tell me a funny joke about a dog'

In [124]:
#  f-string default
from langchain.prompts import PromptTemplate

fstring_template = """Tell me a {adjective} joke about {content}"""
prompt = PromptTemplate.from_template(fstring_template)

prompt.format(adjective="funny", content="chickens")
# Output: Tell me a funny joke about chickens.

'Tell me a funny joke about chickens'

# Types of MessagePromptTemplate

In [128]:
# custom roles

from langchain.prompts import ChatMessagePromptTemplate

prompt = "May the {subject} be with you"

chat_message_prompt = ChatMessagePromptTemplate.from_template(role="Pakistani", template=prompt)
print(chat_message_prompt.format(subject="Pak Army").content)

May the Pak Army be with you


In [129]:
from langchain.prompts import MessagesPlaceholder

human_prompt = "Summarize our convesation so far in {word_count} words."
human_prompt_template = HumanMessagePromptTemplate.from_template(human_prompt)
chat_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="conversation"), human_prompt_template
])

In [142]:
from langchain.schema import HumanMessage,AIMessage

In [151]:
human_message = HumanMessage(content="What is the best way to lean programming?")
ai_message = AIMessage(content="""\
1. Choose a programming language: Decide on a programming language that you want to learn.

2. Start with the basics: Familiarize yourself with the basic programming concepts such as variables, data types and control structures.

3. Practice, practice, practice: The best way to learn programming is through hands-on experience\
""")

abc = chat_prompt.format_prompt(conversation=[human_message, ai_message], word_count=100)

In [153]:
abc.to_messages()

[HumanMessage(content='What is the best way to lean programming?', additional_kwargs={}, example=False),
 AIMessage(content='1. Choose a programming language: Decide on a programming language that you want to learn.\n\n2. Start with the basics: Familiarize yourself with the basic programming concepts such as variables, data types and control structures.\n\n3. Practice, practice, practice: The best way to learn programming is through hands-on experience', additional_kwargs={}, example=False),
 HumanMessage(content='Summarize our convesation so far in 100 words.', additional_kwargs={}, example=False)]

# Partial prompt tempaltes

### Partial with Strings

In [154]:
from langchain.prompts import PromptTemplate

In [155]:
prompt = PromptTemplate(template="{foo}{bar}", input_variables=["foo", "bar"])

In [156]:
partial_prompt = prompt.partial(foo="foo")

In [158]:
partial_prompt.format(bar='basz')

'foobasz'

In [161]:
prompt = PromptTemplate(template="{foo} {bar}", input_variables=["bar"], partial_variables={"foo": "foppz"})
print(prompt.format(bar="boppz"))

foppz boppz


### Partial with functions

In [162]:
from datetime import datetime

def _get_datetime():
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

In [163]:
prompt = PromptTemplate(
    template="Tell me a {adjective} joke about the day {date}",
    input_variables=["adjective", "date"],
)

In [201]:
partial_prompt = prompt.partial(date=_get_datetime)

In [202]:

print(partial_prompt.format(adjective='awesome'))

Tell me a awesome joke about the day 2023-08-13 04:09:47


In [204]:
partial_prompt = prompt.partial(adjective='aswesome')

In [206]:
print(partial_prompt.format(date=_get_datetime))

Tell me a aswesome joke about the day <function _get_datetime at 0x7f460f63be20>


# Composition

In [1]:
from langchain.prompts.pipeline import PipelinePromptTemplate
from langchain.prompts import PromptTemplate

In [2]:
full_template = """{introduction}

{example}

{start}"""

full_prompt = PromptTemplate.from_template(full_template)

In [3]:
introduction_template = """You are impersonating {person}."""
introduction_prompt = PromptTemplate.from_template(introduction_template)

In [4]:
example_template = """Here's an example of an interaction: 

Q: {example_q}
A: {example_a}"""
example_prompt = PromptTemplate.from_template(example_template)

In [5]:
start_template = """Now, do this for real!

Q: {input}
A:"""
start_prompt = PromptTemplate.from_template(start_template)

In [7]:
input_prompts = [
    ("introduction",introduction_prompt),
    ("example", example_prompt),
    ("start", start_prompt),
]

In [8]:
pipeline_prompt = PipelinePromptTemplate(
    final_prompt=full_prompt,
    pipeline_prompts=input_prompts,
)

In [10]:
pipeline_prompt.input_variables

['example_a', 'input', 'example_q', 'person']

In [11]:
print(pipeline_prompt.format(
    person="Elon Musk",
    example_q="What is your favorite car?",
    example_a="Tesla Model S",
    input="what is your favorite social media platform?",
)
)

You are impersonating Elon Musk.

Here's an example of an interaction: 

Q: What is your favorite car?
A: Tesla Model S

Now, do this for real!

Q: what is your favorite social media platform?
A:


# Serialization

In [13]:
# All prompts are loaded through the `load_prompts` function.

from langchain.prompts import load_prompt

### Loading from YAML

In [18]:
prompt = load_prompt("assets/simple_prompt.yaml")

In [25]:
prompt.format(adjective='furry', content='walrus')

'Tell me a furry joke about walrus.'

### Loading from JSON

In [26]:
prompt = load_prompt("assets/simple_prompt.json")
prompt.format(adjective='furry', content='a cat')

'Tell me a furry joke about a cat.'

### Loading Template from a file

In [28]:
prompt = load_prompt("assets/simple_prompt_with_template_file.json")

In [29]:

print(prompt.format(adjective="funny", content="chickens"))

    Tell me a funny joke about chickens.


## FewShotPromptTemplate

# Prompt Pipelining

## String Prompt Pipelining

In [30]:
from langchain.prompts import PromptTemplate

In [31]:
prompt = (
    PromptTemplate.from_template(
        "Tell me a joke about {topic}.")
    + ", make it funny"
    + "\n\nand in {language}"
    
)

In [32]:
prompt.format(topic="sports", language="spanish")

'Tell me a joke about sports., make it funny\n\nand in spanish'

In [33]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

In [34]:
model = ChatOpenAI()

In [35]:
chain = LLMChain(llm=model, prompt=prompt)

In [36]:
chain.run(topic="sports", language="spanish")

'¿Por qué los futbolistas siempre están en forma? \nPorque siempre hacen ejercicio: ¡corren detrás del balón y luego corren detrás del árbitro!'

## Chat Prompt Pipelining

In [37]:
from langchain.prompts import ChatMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.schema import HumanMessage, AIMessage, SystemMessage

In [38]:
prompt = SystemMessage(content="You are a nice pirate")

In [39]:
new_prompt = (
    prompt
    + HumanMessage(content="hi")
    + AIMessage(content="what?")
    + "{input}"
)

In [40]:
new_prompt.format_messages(input="i said hi")

[SystemMessage(content='You are a nice pirate', additional_kwargs={}),
 HumanMessage(content='hi', additional_kwargs={}, example=False),
 AIMessage(content='what?', additional_kwargs={}, example=False),
 HumanMessage(content='i said hi', additional_kwargs={}, example=False)]

In [41]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

In [42]:
model = ChatOpenAI()

In [43]:
chain = LLMChain(llm=model, prompt=new_prompt)

In [44]:
chain.run("i said hi")

'Hello! How can I help you today?'

# Validate template

In [46]:
template = "I am learning langchain because {reason}."

# prompt_template = PromptTemplate(template=template,
#                                  input_variables=["reason", "foo"])

prompt_template = PromptTemplate(template=template,
                                 input_variables=["reason", "foo"],
                                 validate_template=False)

# Example Selectors

In [5]:
from typing import Dict, List
from abc import ABC, abstractmethod

In [6]:
class BaseExampleSelector(ABC):
    """Interface for selecting examples to include in the prompts."""
    
    @abstractmethod
    def select_examples(self, input_variables: Dict[str, str]) -> List[dict]:
        """Select which examples to use based on the inputs."""

# Custom example selector

### Implement custom example selector

In [7]:
from config import *

In [8]:
from langchain.prompts.example_selector.base import BaseExampleSelector
from typing import Dict, List
import numpy as np

In [21]:
from typing import Any, Dict, List


class CustomExampleSelector(BaseExampleSelector):
    
    def __init__(self, example: List[Dict[str, str]]):
        self.examples = examples
        
    def add_example(self, example: Dict[str, str]) -> None:
        self.examples.append(example)
        
    def select_examples(self, input_variables: Dict[str, str]) -> List[dict]:
        return np.random.choice(self.examples, 2, replace=False)

### Using the custom example selector

In [10]:
examples = [
    {"foo": "1"},
    {"foo": "2"},
    {"foo": "3"},
]

example_selector = CustomExampleSelector(examples)


In [17]:
example_selector.select_examples({'',''})

array([{'foo': '2'}, {'foo': '1'}, {'foo': '3'}], dtype=object)

In [19]:
example_selector.add_example({"foo": "4"})
example_selector.examples

[{'foo': '1'}, {'foo': '2'}, {'foo': '3'}, {'foo': '4'}, {'foo': '4'}]

In [28]:
example_selector.select_examples({"":""})

array([{'foo': '3'}, {'foo': '4'}, {'foo': '1'}], dtype=object)

### Select by length

In [35]:
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

In [36]:
examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

In [37]:
example_prompt = PromptTemplate(
    template="Input: {input}\nOutput: {output}\n",
    input_variables=['input', 'output'],
)

In [55]:
example_selector = LengthBasedExampleSelector(
    examples=examples,
    example_prompt=example_prompt,
    max_length=35,
)

In [56]:
dynamic_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

In [57]:
print(dynamic_prompt.format(adjective='big'))

Give the antonym of every input

Input: happy
Output: sad


Input: tall
Output: short


Input: energetic
Output: lethargic


Input: sunny
Output: gloomy


Input: windy
Output: calm


Input: big
Output:


In [58]:
# An example with long input, so it selects only one example.
long_string = "big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else"
print(dynamic_prompt.format(adjective=long_string))

Give the antonym of every input

Input: happy
Output: sad


Input: tall
Output: short


Input: big and huge and massive and large and gigantic and tall and much much much much much bigger than everything else
Output:


In [61]:
# You can add an example to an example selector as well.
new_example =  {"input": "big", "output":"small"}
dynamic_prompt.example_selector.add_example(new_example)
print(dynamic_prompt.format(adjective="expensive"))

Give the antonym of every input

Input: happy
Output: sad


Input: tall
Output: short


Input: energetic
Output: lethargic


Input: sunny
Output: gloomy


Input: windy
Output: calm


Input: big
Output: small


Input: expensive
Output:


# Select by maximal marginal relevance (MMR)

it does this by finding the examples with the embeddings that have the greatest cosine similarity with the inputs, and then iteratively adding them while penalizing them for closeness to already selected examples.

In [63]:
# examples with the embeddings that have the greatest cosine similarity with the inputs
from langchain.prompts.example_selector import (
    MaxMarginalRelevanceExampleSelector,
    SemanticSimilarityExampleSelector
)
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

In [77]:
example_prompt = PromptTemplate(
    template="Input: {input}\nOutput: {output}\n",
    input_variables=["input", "output"],
)

examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "pleasant", "output": "unpleasant"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

In [78]:
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    FAISS,
    k=2,
)

In [79]:
mmr_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)

In [82]:
print(mmr_prompt.format(adjective="pleasent"))

Give the antonym of every input

Input: pleasant
Output: unpleasant


Input: windy
Output: calm


Input: pleasent
Output:


In [76]:
# Let's compare this to what we would just get if we went solely off of similarity,
# by using SemanticSimilarityExampleSelector instead of MaxMarginalRelevanceExampleSelector.

example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    FAISS,
    k=2,
)

similar_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give the antonym of every input",
    suffix="Input: {adjective}\nOutput:",
    input_variables=["adjective"],
)
print(similar_prompt.format(adjective="worried"))

Give the antonym of every input

Input: happy
Output: sad


Input: sunny
Output: gloomy


Input: worried
Output:


# Select by n-gram overlap

In [2]:
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.prompts.example_selector.ngram_overlap import NGramOverlapExampleSelector

In [3]:
# These are examples of a fictional translation task.
examples = [
    {"input": "See Spot run.", "output": "Ver correr a Spot."},
    {"input": "My dog barks.", "output": "Mi perro ladra."},
    {"input": "Spot can run.", "output": "Spot puede correr."},
]

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)

In [16]:
example_selector = NGramOverlapExampleSelector(
    examples = examples,
    example_prompt = example_prompt,
    threshold = 20,
)

In [17]:
dynamic_prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = example_prompt,
    prefix = "Give the Spanish translation of every input",
    suffix = "Input: {sentence}\nOutput:",
    input_variables = ["sentence"],
)

In [18]:
print(dynamic_prompt.format(sentence="Spot can run fast"))

Give the Spanish translation of every input

Input: Spot can run fast
Output:


In [12]:
new_example = {"input":"Spot plays fetch.", "output":"Spot juega a buscar."}

example_selector.add_example(new_example)

In [15]:
print(dynamic_prompt.format(sentence="Spot fetch."))

Give the Spanish translation of every input

Input: Spot plays fetch.
Output: Spot juega a buscar.

Input: See Spot run.
Output: Ver correr a Spot.

Input: Spot can run.
Output: Spot puede correr.

Input: Spot fetch.
Output:


#  Select by similarity

In [24]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

In [22]:
example_prompt = PromptTemplate(
    input_variables = ["input","output"],
    template = "Input: {input}\nOutput: {output}\n\n"
)

examples = [
    {"input": "happy", "output": "sad"},
    {"input": "tall", "output": "short"},
    {"input": "energetic", "output": "lethargic"},
    {"input": "sunny", "output": "gloomy"},
    {"input": "windy", "output": "calm"},
]

In [31]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples,
    OpenAIEmbeddings(),
    Chroma,
    k=1,
)

similar_prompt = FewShotPromptTemplate(
    example_selector = example_selector,
    example_prompt = example_prompt,
    prefix = "Give the antonym of every input",
    suffix = "Input: {adjective}\nOutput:",
    input_variables = ["adjective"],
)

In [34]:
print(similar_prompt.format(adjective='worried'))

Give the antonym of every input

Input: happy
Output: sad



Input: worried
Output:


In [35]:
# You can add new examples to the SemanticSimilarityExampleSelector as well
similar_prompt.example_selector.add_example({"input": "enthusiastic", "output": "apathetic"})
print(similar_prompt.format(adjective="joyful"))

Give the antonym of every input

Input: happy
Output: sad



Input: joyful
Output:
